# Problema do Caixeiro Viajante(TSP) com ACO
Na indiústria de equipamentos eletrônicos há um problema bastante usual que é a determinação da sequência de furação de uma placa de circuito impresso (*Printed Circuit Board* - PCB). Para agilizar a produção em massa de placas é necessário que o processo de furação seja o mais rápido possível e em uma placa com alta densidade de furos, o somatório dos tempos de deslocamento da furadeira entra cada furo pode ser muito significativo. Assim, é necessário otimizar o trajeto da furadeira de maneira tal a apassar poro todos os furos uma única vez com o menor deslocamento possível.  

Este problema real pode ser tratado como o problema clássico do Caixeiro Viajante(TSP) simétrico(a distância entre os pontos não depende do sentido de deslocamento).  

Com base na planilha que contém a localização de cada furo(**planilha-pcb-2017.xls**)será implementado em Python (https://github.com/ppoffice/ant-colony-tsp)

In [19]:
%matplotlib notebook
import math
import numpy as np
import time as time
from aco import ACO, Graph
from plot import plot
from random import shuffle


Importando os dados do arquivo para gerar o mundo que o algortimo se propoe a resolver.  
Esse mundo é composto dos *nodes*(furos da placa) e da função de custo que deve aceitar dois nodes e calcular o custo associado a eles.

In [20]:
#Leitura dos dados
nodes = np.genfromtxt('planilha-pcb-2017.txt',delimiter=',').tolist()
shuffle(nodes)

#Tamanho do set
rank = len(nodes)

#Função de custo é a própria distância euclideana
def euclidean(a, b):
    return math.sqrt(pow(a[1] - b[1], 2) + pow(a[0] - b[0], 2))

A lib utilizada para esse experimento utiliza um grafo com os custos. É um grafo totalemtente conectado, ou seja, todos os pontos se conectam com todos os pontos. A seguir fazemos a carga desse grafo

In [21]:
cost_matrix = [[euclidean(i,j) for i in nodes ]for j in nodes]
graph = Graph(cost_matrix, rank)

A seguir intanciamos e executamos uma configuração do ACO

In [43]:
start_time = time.time()
"""
:param ant_count:
:param generations:
:param alpha: relative importance of pheromone
:param beta: relative importance of heuristic information
:param rho: pheromone residual coefficient
:param q: pheromone intensity
:param strategy: pheromone update strategy. 0 - ant-cycle, 1 - ant-quality, 2 - ant-density
"""
aco = ACO(ant_count=30, generations=100, alpha=1.0, beta=10.0, rho=0.1, q=10, strategy=2)
path, cost = aco.solve(graph)
end_time = time.time() - start_time

C:\repo\MESTRADO\notebooks\computacao_evolucionaria\aco.py:93: RuntimeWarning: invalid value encountered in double_scalars
  self.eta[self.current][i] ** self.colony.beta / denominator


ValueError: list.remove(x): x not in list

In [ ]:
print('cost: {}, time: {}, path: {}'.format(cost, end_time,  path))
plot(nodes, path)

Com o intuito de estudar o comportamento do ACO para esse problema, abaixo variamos alguns parâmetros do ACO.  

## Tamanho da população
Variou-se o tamanho da população e um intervalo determinado para identificar o melhor distancia e o tempo gasto para rodar o algoritmo.


In [32]:
N = np.arange(1,101,5)
columns = "{!s:<25}\t{:<50}\t{:<25}"
divider = "-" * (25 + 50 + 25)
header = columns.format("Formigas","Tempo", "Distancia")
columns = columns.replace('<', '>', 1)

print()
print(header)
print(divider)

results_pop = []
for n in N:
    start_time = time.time()
    aco = ACO(ant_count=n, generations=20, alpha=1.0, beta=5.0, rho=0.8, q=10, strategy=2)
    path, cost = aco.solve(graph)
    end_time = time.time() - start_time
    print(columns.format(n, end_time, cost))
    results_pop.append([n,end_time,cost,path])


Formigas                 	Tempo                                             	Distancia                
----------------------------------------------------------------------------------------------------
                        1	49.162729024887085                                	2000.1068064200344       
                        6	169.69266867637634                                	1922.8243075902585       
                       11	245.4088637828827                                 	1899.3629054065266       
                       16	307.514447927475                                  	1881.0471715924054       
                       21	388.67637372016907                                	1905.892554594869        
                       26	479.2900285720825                                 	1879.3715267446894       
                       31	569.8003523349762                                 	1843.0496646140853       
                       36	675.6660778522491                               

## Gerações

In [33]:
N = np.arange(1,101,5)
columns = "{!s:<25}\t{:<50}\t{:<25}"
divider = "-" * (25 + 50 + 25)
header = columns.format("Gerações","Tempo", "Distancia")
columns = columns.replace('<', '>', 1)

print()
print(header)
print(divider)

results_ger = []
for n in N:
    start_time = time.time()
    aco = ACO(ant_count=10, generations=n, alpha=1.0, beta=5.0, rho=0.8, q=10, strategy=2)
    path, cost = aco.solve(graph)
    end_time = time.time() - start_time
    print(columns.format(n, end_time, cost))
    results_ger.append([n,end_time,cost,path])


Gerações                 	Tempo                                             	Distancia                
----------------------------------------------------------------------------------------------------
                        1	9.205644369125366                                 	1972.214494251884        
                        6	55.70988941192627                                 	1915.7527527364714       
                       11	102.95006322860718                                	1890.9280639220142       
                       16	149.76768517494202                                	1912.982757332536        
                       21	194.30582332611084                                	1882.801402291658        
                       26	240.7142686843872                                 	1913.3715096095837       
                       31	286.43787574768066                                	1892.5127637039834       
                       36	334.77527809143066                              

## Tempo do feromonio

In [34]:
N = np.arange(0.1,1.1,0.1)
columns = "{!s:<25}\t{:<50}\t{:<25}"
divider = "-" * (25 + 50 + 25)
header = columns.format("Evap. Feromonio","Tempo", "Distancia")
columns = columns.replace('<', '>', 1)

print()
print(header)
print(divider)

results_fer = []
for n in N:
    start_time = time.time()
    aco = ACO(ant_count=10, generations=10, alpha=1.0, beta=5.0, rho=n, q=10, strategy=2)
    path, cost = aco.solve(graph)
    end_time = time.time() - start_time
    print(columns.format(n, end_time, cost))
    results_fer.append([n,end_time,cost,path])


Evap. Feromonio          	Tempo                                             	Distancia                
----------------------------------------------------------------------------------------------------
                      0.1	100.85037875175476                                	1884.6102761078441       
                      0.2	104.26238203048706                                	1898.516768619044        
                      0.3	101.92535424232483                                	1903.9737344665102       
                      0.4	100.59638381004333                                	1901.2895620166553       
                      0.5	102.1123833656311                                 	1875.922277583089        
                      0.6	102.67437100410461                                	1905.5017847083423       
                      0.7	100.22438311576843                                	1934.498894056896        
                      0.8	102.654372215271                                

## Alpha X Beta

In [39]:
N = np.arange(1,11,2)
columns = "{!s:<20}\t{:<20}\t{:<50}\t{:<25}"
divider = "-" * (20 + 20 +50 + 25)
header = columns.format("Alpha", "Beta", "Tempo", "Distancia")
columns = columns.replace('<', '>', 1)

print()
print(header)
print(divider)

results_ab = []
for a in N:
    for b in N:
        start_time = time.time()
        aco = ACO(ant_count=10, generations=10, alpha=a, beta=b, rho=0.8, q=10, strategy=2)
        path, cost = aco.solve(graph)
        end_time = time.time() - start_time
        print(columns.format(a, b, end_time, cost))
        results_ab.append([a,b,end_time,cost,path])


Alpha               	Beta                	Tempo                                             	Distancia                
-------------------------------------------------------------------------------------------------------------------
                   1	1                   	167.23805332183838                                	2126.902615543122        
                   1	3                   	160.29008436203003                                	2039.3297871124537       
                   1	5                   	167.22299027442932                                	1977.6466707384218       
                   1	7                   	159.13943934440613                                	1942.3332410766805       
                   1	9                   	166.02905559539795                                	1946.0427967484625       


C:\repo\MESTRADO\notebooks\computacao_evolucionaria\aco.py:93: RuntimeWarning: invalid value encountered in double_scalars
  self.eta[self.current][i] ** self.colony.beta / denominator


ValueError: list.remove(x): x not in list

## Melhor Solução

In [56]:
OFFSET = 2
pathX,pathY = [s[0] for s in solution.tour],[s[1] for s in solution.tour]
def data_gen(t=0):
    cnt = 0
    while cnt < len(solution.tour):
        cnt += 1
        yield pathX[cnt], pathY[cnt]


def init():
    ax.set_ylim(min(pathY)-OFFSET, max(pathY)+OFFSET)
    ax.set_xlim(min(pathX)-OFFSET, max(pathX)+OFFSET)
    del xdata[:]
    del ydata[:]
    line.set_data(xdata, ydata)
    return line,

fig, ax = plt.subplots()
line, = ax.plot([], [], lw=2)
ax.grid()
xdata, ydata = [], []


def run(data):
    # update the data
    t, y = data
    xdata.append(t)
    ydata.append(y)
    xmin, xmax = ax.get_xlim()

    if t >= xmax:
        ax.set_xlim(xmin, 2*xmax)
        ax.figure.canvas.draw()
    line.set_data(xdata, ydata)

    return line,

ani = animation.FuncAnimation(fig, run, data_gen, blit=False, interval=100,
                              repeat=True, init_func=init)
plt.show()